# Wikipedia Notable Life Expectancies

# [Notebook 4 of 4: Data Cleaning](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean2_thanak_2022_06_17.ipynb)

## Context

The


## Objective

The

### Data Dictionary

- Feature: Description

## Importing Necessary Libraries

In [1]:
# To structure code automatically
%load_ext nb_black

# To import/export sqlite databases
import sqlite3 as sql

# To help with reading, cleaning, and manipulating data
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)

<IPython.core.display.Javascript object>

## Data Overview

### Reading, Sampling, and Checking Data Shape

In [ ]:
# Reading the dataset
conn = sql.connect("wp_life_expect_clean2.db")
data = pd.read_sql("SELECT * FROM wp_life_expect_clean2", conn)

# Making a working copy
df = data.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

In [ ]:
# Checking last 2 rows of the data
df.tail(2)

In [ ]:
# Checking a sample of the data
df.sample(5)

### Checking Data Types, Duplicates, and Null Values

In [ ]:
# Checking data types and null values
df.info()

## Extracting Nationality Continued

#### Checking First Word in `info_1` for `nation_1` Values

In [ ]:
# Column to check
column = "info_1"

# Extract-to column
extract_to = 'nation_1'

# Dataframe to check
dataframe = df[df[column].notna()]

# For loop to split each value into a list and check if first word is in nation_lst
# and assign country to new nation_1 column if found.  Removes found value from column searched.
for index in dataframe.index:
    item = df.loc[index, column]
    split_1 = item.split()
    first_word = split_1[0].strip()
    if first_word in nation_country_dict:
        df.loc[index, extract_to] = nation_country_dict[first_word]
        df.loc[index, column] = df.loc[index, column].replace(first_word, "")

# Check number and sample of treated rows
print(f'{len(df[df[extract_to].notna()])} rows were treated.')
df[df[extract_to].notna()].sample(2)

#### Observations:
- In a way, we are fortunate to have a small sample of rows on which to test code in `info_1`.
- We successfully extracted those `nation_1` values, now we will do the same on the treated rows for `nation_2`.

#### Checking First Word in `info_1` for `nation_2` Values